In [7]:
%load_ext autoreload
%autoreload 2
from config import df_ba_ratings, df_rb_ratings
from src.models.seasonality_analysis import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
plot_beer_style_ranking_by_amount_subplot(df_rb_ratings, df_rb_ratings['style'].unique(), 300, 14)

We lost 133 rows by filtering out styles with less than 300 reviews.
